ABW Assignment 1: Feed Calculator

In [1]:
import sys
at_colab = "google.colab" in sys.modules
if at_colab:
    import shutil
    if not shutil.which("pyomo"):
        !pip install -q pyomo
        assert(shutil.which("pyomo"))
if at_colab:
    if not shutil.which('/usr/bin/glpsol'):
        !sudo apt install libglpk-dev python3.8-dev libgmp3-dev
        !apt-get install -y -qq glpk-utils
        assert(shutil.which('/usr/bin/glpsol'))

    if not shutil.which('/usr/bin/cbc'):
        !apt-get install -y -qq coinor-cbc
        assert(shutil.which('/usr/bin/cbc'))

import matplotlib.pyplot as plt
import math
import numpy as np
import pandas as pd
import pyomo.environ as pyo

import sys
if 'google.colab' in sys.modules:
    import os
    from google.colab import files
    # just check if we already uploaded, may we restart the runtime and run all cells
    if not os.path.isfile('Data Set Feedcalculator.xlsx'):
        uploaded = files.upload()

#import and read data file 
data = pd.read_excel('Data Set Feedcalculator.xlsx',sheet_name=None)

#create data frames of data in excel
ingredients      = data['Ingredient Database']
nutrient_rules   = data['Nutrient Rules']
ingredient_rules = data['Ingredient Rules']

#create dataframe with all available nutrients
available_ingredients = ingredients[ingredients.Availability][['Name','Reference name','Price']+list(nutrient_rules.Nutrient)].set_index('Reference name')

#create a dictionary with keys: name of the nutrient and values: lower and upper bound of the nutrient
nutrient_bounds = { nut : (lb if not pd.isna(lb) else 0,     # no lower bound translates into 0 as lower bound
                           ub if not pd.isna(ub) else None)  # no upper bound becomes None
                    for nut,lb,ub in zip(nutrient_rules.Nutrient,nutrient_rules['Lower Bound'],nutrient_rules['Upper Bound']) 
                  }

ingredient_bounds = { ing : (lb if not pd.isna(lb) else 0,    # no lower bound translates into 0 as lower bound
                             ub if not pd.isna(ub) else None) # no upper bound becomes None
                      for ing,lb,ub in zip(ingredient_rules.Ingredient,ingredient_rules['Lower Bound'],ingredient_rules['Upper Bound']) }


combined_ingredient_rules   = [] 
set_of_availabe_ingredients = set(available_ingredients.index)
for c in ['Unnamed: '+str(i) for i in range(5,13)]:
    aux = ingredient_rules[[c]].dropna().values
    aux = [ v[0] for v in aux ]
    upperbound          = aux[0]
    ingredients_in_rule = set(aux[1:]).intersection(set_of_availabe_ingredients)
    if ingredients_in_rule:
        combined_ingredient_rules.append((upperbound,ingredients_in_rule))
combined_ingredient_rules

def FeedCalculator(available_ingredients, ingredient_bounds, combinded_ingredient_rules):

  model = pyo.ConcreteModel('FeedCalculator')

  model.ingredients = pyo.Set(initialize=available_ingredients.index)
  model.nutrients = pyo.Set(initialize=nutrient_bounds.keys())
  model.cir = pyo.Set(initialize = list(range(len(combined_ingredient_rules))))

  def bounds_rule(model, i):
    return ingredient_bounds.get(i, (0,None))
  
  model.x = pyo.Var(model.ingredients, bounds=bounds_rule, within = pyo.NonNegativeReals)

  model.optimal = pyo.Objective(expr = pyo.quicksum(available_ingredients.Price[i]*model.x[i] for i in model.ingredients), sense = pyo.minimize)

  #add constraint 1: weight of food has to qual one
  model.weight = pyo.Constraint(expr = pyo.quicksum(model.x[i] for i in model.ingredients) == 1)

  #add constraint 2: nutritional requirements have to be fulfilled
  def nutrient_rule(model, n):
    l, u= nutrient_bounds.get(n, (0, None))
    return (l, pyo.quicksum(model.x[i]*available_ingredients.loc[i][n] for i in model.ingredients), u)
  model.nutrition = pyo.Constraint(model.nutrients, rule = nutrient_rule)
   

  def combined_rule(model, r):
    return (pyo.quicksum(model.x[i] for i in combined_ingredient_rules[r][1]) <= combined_ingredient_rules[r][0])
  model.combined = pyo.Constraint(model.cir, rule = combined_rule)

  return model

FeedCalculator = FeedCalculator(available_ingredients, ingredient_bounds, nutrient_bounds)
FeedCalculator.pprint()

%time results=pyo.SolverFactory('cbc').solve(FeedCalculator)
print(results.solver.status, results.solver.termination_condition)
print(FeedCalculator.optimal.expr())

FeedCalculator.display()

     |████████████████████████████████| 9.1 MB 7.2 MB/s 
     |████████████████████████████████| 49 kB 3.4 MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libamd2 libbtf1 libcamd2 libccolamd2 libcholmod3 libcolamd2 libcxsparse3
  libglpk40 libgmp-dev libgmpxx4ldbl libgraphblas1 libklu1 libldl2 libmetis5
  libpython3.8 libpython3.8-dev libpython3.8-minimal libpython3.8-stdlib
  librbio2 libspqr2 libsuitesparse-dev libsuitesparseconfig5 libumfpack5
  python3.8 python3.8-minimal
Suggested packages:
  libiodbc2-dev gmp-doc libgmp10-doc libmpfr-dev python3.8-venv python3.8-doc
  binfmt-support
The following NEW packages will be installed:
  libamd2 libbtf1 libcamd2 libccolamd2 libcholmod3 libcolamd2 libcxsparse3
  libglpk-dev libglpk40 libgmp-dev libgmp3-dev libgmpxx4ldbl libgraphblas1
  libklu1 libldl2 libmetis5 libpython3.8 libpython3.8-dev libpython3.8-minimal
  libpython3.8-stdli

Saving Data Set Feedcalculator.xlsx to Data Set Feedcalculator.xlsx
3 Set Declarations
    cir : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    7 : {0, 1, 2, 3, 4, 5, 6}
    ingredients : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   26 : {'barley', 'blood', 'boneash', 'cotton', 'fish', 'fishlq', 'gnseeds', 'maize', 'maizebranhighq', 'maizebranlowq', 'mbmeal', 'sugars', 'soybeanexp', 'soybeanmeal', 'sunflower', 'sunflowerseeds', 'tapbran', 'caswhole', 'casfine', 'wheatbran', 'lysine', 'dl', 'ltryp', 'dicaph', 'shells', 'salt'}
    nutrients : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   14 : {'oebr', 'cp', 'cfibre', 'staew', 'ca', 'na', 'opp', 'dlysp', 'dmetp', 'dmcp', 'dthrp', 'dtryp', 'dvalp', 'dargp'}

1 Var Declarations
    x : Size=26, Index=ingredients
        Key